In [15]:
%load_ext nb_black

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport

import altair as alt

from scipy import stats
from scipy.stats import norm, skew
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn


import os

os.chdir('/Users/66789/Documents/GitHub/aag-bain-f2fw/')
datafile = 'data/230428 trucking2303_shared with AAG_color.xlsx'

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## Reading the Data

In [20]:
freq = "QUARTERLY"
df = pd.read_excel(datafile, sheet_name=freq).T
new_header = df.iloc[1] #grab the first row for the header
df = df[3:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.drop(['Label: (SA = seasonally adjusted)'], axis=1, inplace=True)
df = df.loc[:, df.columns.notna()]
df = df.loc[:, ~df.columns.str.startswith('TCI Component')]
df.dropna(axis = 0, how = 'all', inplace = True)

df = df.apply(pd.to_numeric, downcast='float', errors='coerce')

if freq == "QUARTERLY":
    df['Date'] = pd.date_range('2000-01-01', '2025-12-01', freq='QS')
    df['Year'] = df['Date'].dt.year
    df['Quarter'] = df['Date'].dt.quarter
else:
    df['Date'] = pd.date_range('2000-01-01', '2025-01-01', freq='MS')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month


display(df.describe())



Unnamed: 1,Active Truck Utilization (SA),Total Truck Loadings (SA),"Total TL: Spot Rate (exc. FSC, SA)","Total TL: Contract Rate (exc. FSC, SA)","Driver Labor Index (1992=100, SA)","Truck Driver Pressure Index (0 = Neutral, SA)",Real GDP,CPI Index,"3 Month T-Bill Rate, %",National Avg. Diesel Fuel Price ($/Gal.),Year,Quarter
count,104.000000,1.040000e+02,72.000000,72.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.00000,104.000000
mean,0.909791,1.739228e+08,112.676071,120.851532,120.870682,0.087948,16797.249586,2.342130,1.919127,2.975320,2012.50000,2.500000
std,0.044867,1.577288e+07,17.677216,18.209970,7.377230,5.701280,2332.830338,0.431301,1.970776,0.997807,7.53632,1.123448
min,0.818753,1.421712e+08,77.902023,95.654587,105.248306,-11.974460,12935.252000,1.701000,0.013333,1.178333,2000.00000,1.000000
25%,0.878123,1.626295e+08,103.122383,105.698410,113.009853,-3.726186,15209.354000,2.020667,0.123333,2.368833,2006.00000,1.750000
50%,0.902286,1.715567e+08,108.940704,114.899826,124.596096,-1.007951,16370.760000,2.317072,1.201667,2.919000,2012.50000,2.500000
75%,0.936137,1.909591e+08,119.990997,139.079742,125.628685,4.476465,18766.643066,2.555185,3.501532,3.889917,2019.00000,3.250000
max,1.000000,1.992374e+08,162.431305,156.782104,130.137955,11.560026,21040.896484,3.358075,6.200000,5.481667,2025.00000,4.000000


<IPython.core.display.Javascript object>

## EDA
* Missing values
* Correlations
* Time series analysis

In [21]:
train_missing = (df.isnull().sum()/len(df)) *100
train_missing = train_missing.drop(train_missing[train_missing ==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Percentage': train_missing})
missing_data

,Missing Percentage
Unnamed: 1,
"Total TL: Spot Rate (exc. FSC, SA)",30.769231
"Total TL: Contract Rate (exc. FSC, SA)",30.769231


<IPython.core.display.Javascript object>

In [23]:
# timeseries_heatmap(dataframe=df, entity_column='Site Num', sortby='Date')

profile = ProfileReport(df, tsmode=True, sortby="Date")
profile.to_file(freq+'_profile_report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.35it/s]


<IPython.core.display.Javascript object>

In [ ]:
train_spot = df[['Date', 'Total TL: Spot Rate (exc. FSC, SA)']].dropna()
train_spot.set_index('Date', inplace=True)
train_spot.index.freq='QS'
train_spot.index

train_contract = df[['Date', 'Total TL: Contract Rate (exc. FSC, SA)']].dropna()
train_contract.set_index('Date', inplace=True)
train_contract.index.freq='QS'
train_contract.index


In [ ]:
print('Contract rate')
sns.heatmap(pd.pivot_table(data=train_contract, index=train_contract.index.year, columns=train_contract.index.quarter),
           square=True,
           cmap='Blues',
           xticklabels=["Q1", "Q2", "Q3", "Q4"])



In [ ]:
print('Spot rate')
sns.heatmap(pd.pivot_table(data=train_spot, index=train_spot.index.year, columns=train_spot.index.quarter),
           square=True,
           cmap='Blues',
           xticklabels=["Q1", "Q2", "Q3", "Q4"])

In [ ]:
sns.pairplot(df)

In [ ]:
pd.plotting.lag_plot(train_spot["Total TL: Spot Rate (exc. FSC, SA)"],1)

In [ ]:
pd.plotting.lag_plot(train_spot["Total TL: Spot Rate (exc. FSC, SA)"],4)